In [1]:
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import statsmodels.formula.api as smf 

In [2]:
print(f'Current working directory is {os.getcwd()}')

Current working directory is /Users/drewrichard/Documents/1 projects/nss/nss_projects/accre-carbonara/notebooks


In [3]:
yellow_color = '#FFD96E'
blue_color = '#3F4C5D'

In [4]:
# Import data
jobs = pd.read_csv("../data/fullsample.csv",
                    # nrows = 10000
                    )
ce5 = pd.read_csv('../data/ce5_unresponsive.csv')
ce6 = pd.read_csv('../data/ce6_unresponsive.csv')
jobs = jobs[jobs['END'] != 'Unknown']

# Convert dates to datetime objects
jobs['END'] = pd.to_datetime(jobs['END'])
jobs['BEGIN'] = pd.to_datetime(jobs['BEGIN'])

# String manipulation of memory columns
jobs['USEDMEM'] = jobs['USEDMEM'].str[:-1]
jobs.insert(5, 'Mc_Mn', jobs['REQMEM'].str[-2:])
jobs['REQMEM']  = jobs['REQMEM'].str[:-2]

# Convert to numeric values for easier manipulation, and create difference col
jobs['USEDMEM'] = pd.to_numeric(jobs['USEDMEM'])
jobs['REQMEM'] = pd.to_numeric(jobs['REQMEM'])
jobs['DIFFMEM'] = jobs['REQMEM'] - jobs['USEDMEM']

# Create boolean column for status = COMPLETE
jobs.insert(2, 'COMPLETE', jobs['STATE'] == 'COMPLETED')
jobs['COMPLETE'] = jobs['COMPLETE'].astype(int)

# Create columns to assign failure, job counts, and concat into one df
ce56 = pd.concat([ce5, ce6])
jobs['FAILED'] = 0
jobs['JOBCOUNT'] = 1
ce56['JOBCOUNT'] = 0
ce56['FAILED'] = 1
jobs_ce56 = pd.concat([jobs, ce56])

# Eliminate milliseconds in desired datetime column 'END' 
jobs_ce56['END'] = pd.to_datetime(jobs_ce56['END']).dt.floor('s')
jobs_ce56.head(5)

,JOBID,STATE,COMPLETE,BEGIN,END,REQMEM,Mc_Mn,USEDMEM,REQTIME,USEDTIME,...,PARTITION,EXITCODE,DIFFMEM,FAILED,JOBCOUNT,USER,RETRY,TIME,RETURNCODE,COMMAND
1,30853133,COMPLETED,1.0,2021-08-06 11:36:09,2021-09-05 11:36:32,262144.0,Mn,20604.62,30-00:00:00,30-00:00:23,...,cgw-platypus,0:0,241539.38,0,1,NaN,NaN,NaN,NaN,NaN
2,30858137,COMPLETED,1.0,2021-08-06 19:04:39,2021-09-05 19:04:53,204800.0,Mn,57553.77,30-00:00:00,30-00:00:14,...,cgw-tbi01,0:0,147246.23,0,1,NaN,NaN,NaN,NaN,NaN
3,30935078,COMPLETED,1.0,2021-08-09 16:52:51,2021-09-07 20:52:55,65536.0,Mn,20577.96,29-04:00:00,29-04:00:04,...,cgw-platypus,0:0,44958.04,0,1,NaN,NaN,NaN,NaN,NaN
4,31364111_2,COMPLETED,1.0,2021-08-17 07:45:07,2021-09-10 16:45:24,16384.0,Mn,9733.43,24-09:00:00,24-09:00:17,...,production,0:0,6650.57,0,1,NaN,NaN,NaN,NaN,NaN
5,31364111_3,COMPLETED,1.0,2021-08-17 07:45:07,2021-09-06 16:17:34,16384.0,Mn,9708.04,24-09:00:00,20-08:32:27,...,production,0:0,6675.96,0,1,NaN,NaN,NaN,NaN,NaN


In [5]:
ce56.shape

(3296, 8)

In [ ]:
# Set the time window in minutes
x = 20
time_delta = pd.Timedelta(minutes=x)

# Extract rows where FAILED == 1
failed_rows = jobs_ce56[jobs_ce56['FAILED'] == 1][['END']].sort_values('END')

# Sort the main DataFrame by 'END'
jobs_ce56 = jobs_ce56.sort_values('END')

# Convert 'END' to numpy arrays for fast computation
end_times = jobs_ce56['END'].to_numpy()
failed_end_times = failed_rows['END'].to_numpy()

# Use NumPy searchsorted to find rows in the time range
start_idx = np.searchsorted(end_times, failed_end_times - time_delta, side='left')
end_idx = np.searchsorted(end_times, failed_end_times, side='right')

# Collect indices for all matching rows
matching_indices = np.concatenate([np.arange(start, end) for start, end in zip(start_idx, end_idx)])

# Extract the matching rows from the original DataFrame
result = jobs_ce56.iloc[np.unique(matching_indices)].copy()

# Identify missing indicators
result['group'] = result['FAILED']==1

# Shift the group assignment up by one
result['group'] = result['group'].shift(-1, fill_value=False).cumsum()

# Display result
# display(result.head(3))

# Identify unique rows based on column 'END'
result = result[~result['END'].duplicated(keep=False)]

display(result.head())

,JOBID,STATE,COMPLETE,BEGIN,END,REQMEM,Mc_Mn,USEDMEM,REQTIME,USEDTIME,...,EXITCODE,DIFFMEM,FAILED,JOBCOUNT,USER,RETRY,TIME,RETURNCODE,COMMAND,group
7054741,25041599,COMPLETED,1.0,2020-10-18 05:39:54,2020-10-18 05:57:39,61440.0,Mn,4450.38,5-00:00:00,00:17:45,...,0:0,56989.62,0,1,NaN,NaN,NaN,NaN,NaN,0
7054510,25041077,COMPLETED,1.0,2020-10-18 05:38:00,2020-10-18 05:58:37,21880.0,Mn,1.26,2-00:00:00,00:20:37,...,0:0,21878.74,0,1,NaN,NaN,NaN,NaN,NaN,0
7051604,25033847_909,COMPLETED,1.0,2020-10-17 20:53:43,2020-10-18 05:58:42,6144.0,Mn,1646.39,8-08:00:00,09:04:59,...,0:0,4497.61,0,1,NaN,NaN,NaN,NaN,NaN,0
7054506,25041073,COMPLETED,1.0,2020-10-18 05:38:00,2020-10-18 05:58:44,21875.0,Mn,1.60,2-00:00:00,00:20:44,...,0:0,21873.40,0,1,NaN,NaN,NaN,NaN,NaN,0
7054508,25041075,COMPLETED,1.0,2020-10-18 05:38:21,2020-10-18 05:59:09,21875.0,Mn,1.80,2-00:00:00,00:20:48,...,0:0,21873.20,0,1,NaN,NaN,NaN,NaN,NaN,0


In [15]:
# Sum the USEDMEM values for each group
totalmem = result.groupby('group')['USEDMEM'].sum()
totalmem = pd.DataFrame(totalmem).reset_index()
totalmem

,group,USEDMEM
0,0,13198.98
1,1,96016.37
2,2,131281.30
3,3,1.63
4,4,172834.66
...,...,...
3041,3292,644456.86
3042,3293,342818.27
3043,3294,290729.62
3044,3295,1469.82


In [16]:
fig = px.scatter(totalmem,
           x=totalmem['group'],
           y=totalmem['USEDMEM'],
           opacity=0.25,
           labels={'group':'Group Number', 'USEDMEM':'Total Used Memory (MB/node)'},
           #title='It is unlikely that memory use is the direct cause of Unresponsive indicator',
           color_discrete_sequence=[yellow_color]
           )
# Calculate average y value
avg_y = totalmem['USEDMEM'].mean()

# Add horizontal line at average y
fig.add_hline(
    y=avg_y,
    line_dash="dot",
    line_color=yellow_color,
    # label=dict(
    #     text=f"mean={avg_y:.2f} MBpN",
    #     textposition='middle',
    #     font=dict(size=18, color='white'),
    #     yanchor="top",
    # ),
)
# Change the background color
fig.update_layout(plot_bgcolor=blue_color,
                      xaxis=dict(showgrid=False),
                      yaxis=dict(showgrid=False),
                      )
fig.write_html('../assets/memoryused.html')
fig.write_image('../assets/memoryused.jpeg')
fig

 The 'textposition' property is an enumeration that may be specified as:
      - One of the following enumeration values:
            ['top left', 'top center', 'top right', 'middle left',
            'middle center', 'middle right', 'bottom left', 'bottom
            center', 'bottom right', 'start', 'middle', 'end']

In [17]:
nodes = result.value_counts('Mc_Mn')
display(nodes)

Mc_Mn
Mn    114443
Mc     31097
Name: count, dtype: int64

In [18]:
totaldiffmem = result.groupby('group')['DIFFMEM'].sum()
totaldiffmem = pd.DataFrame(totaldiffmem).reset_index()
totaldiffmem

,group,DIFFMEM
0,0,539540.02
1,1,512669.63
2,2,481272.70
3,3,21876.37
4,4,933420.34
...,...,...
3041,3292,2128963.14
3042,3293,327313.73
3043,3294,254221.38
3044,3295,26501.18


In [19]:
fig2 = px.scatter(totaldiffmem,
           x=totaldiffmem['group'],
           y=totaldiffmem['DIFFMEM'],
           opacity=0.25,
           labels={'group':'Group Number', 'DIFFMEM':'Difference in memory (MB/node)'},
           #title='It is unlikely that memory use is the direct cause of Unresponsive indicator',
           color_discrete_sequence=[yellow_color]
           )
# Calculate average y value
avg_y = totaldiffmem['DIFFMEM'].mean()

# Add horizontal line at average y
fig2.add_hline(
    y=avg_y,
    line_dash="dot",
    line_color=yellow_color,
    label=dict(
        text=f"average difference in memory used={avg_y:.2f} MBpN",
        textposition='start',
        font=dict(size=15, color=yellow_color),
        yanchor="bottom"
    ),
)
# Change the background color
fig2.update_layout(plot_bgcolor=blue_color,
                      xaxis=dict(showgrid=False),
                      yaxis=dict(showgrid=False))


In [20]:
jobcounts = result.groupby('group')['JOBCOUNT'].sum()
jobcounts = pd.DataFrame(jobcounts).reset_index()
jobcounts

,group,JOBCOUNT
0,0,34
1,1,35
2,2,26
3,3,1
4,4,40
...,...,...
3041,3292,101
3042,3293,150
3043,3294,119
3044,3295,3


In [21]:
fig3 = px.scatter(jobcounts,
           x=jobcounts['group'],
           y=jobcounts['JOBCOUNT'],
           opacity=0.25,
           labels={'group':'Group Number', 'JOBCOUNT':'number of jobs'},
           #title='It is unlikely that number of jobs is the direct cause of Unresponsive indicator',
           color_discrete_sequence=[yellow_color]
           )
# Calculate average y value
avg_y = jobcounts['JOBCOUNT'].mean()

# Add horizontal line at average y
fig3.add_hline(
    y=avg_y,
    line_dash="dot",
    line_color=yellow_color,
    # label=dict(
    #     text=f"average number of jobs={avg_y:.2f}",
    #     textposition='start',
    #     font=dict(size=10, color=yellow_color),
    #     yanchor="top",
    # ),
)
# Change the background color
fig3.update_layout(plot_bgcolor=blue_color,
                      xaxis=dict(showgrid=False),
                      yaxis=dict(showgrid=False))
fig3.add_vrect(x0=500, x1=1200, 
              annotation_text="Very few jobs running", annotation_position="top left",
              annotation=dict(font_color=yellow_color),fillcolor=yellow_color, opacity=0.25, line_width=0)
fig3.write_html('../assets/jobsrunning.html')
fig3.write_image('../assets/jobsrunning.jpeg')
fig3


In [22]:
# Sum Nodes for each group
nodesum = result.groupby('group')['NODES'].sum()
nodesum = pd.DataFrame(nodesum).reset_index()

# Plot sum of nodes used in 20min before failure
fig4 = px.scatter(nodesum,
           x=nodesum['group'],
           y=nodesum['NODES'],
           opacity=0.25,
           labels={'group':'Group Number', 'NODES':'Nodes used'},
           #title='It is unlikely that memory use is the direct cause of Unresponsive indicator',
           color_discrete_sequence=[yellow_color]
           )
# Calculate average y value
avg_y = nodesum['NODES'].mean()

# Add horizontal line at average y
fig4.add_hline(
    y=avg_y,
    line_dash="dot",
    line_color=yellow_color,
    label=dict(
        text=f"average number of nodes used={avg_y:.2f}",
        textposition='start',
        font=dict(size=10, color=yellow_color),
        yanchor="top",
    ),
)
# Change the background color
fig4.update_layout(plot_bgcolor=blue_color,
                      xaxis=dict(showgrid=False),
                      yaxis=dict(showgrid=False))

In [23]:
# Sum the CPU values for each group
cpusum = result.groupby('group')['CPUS'].sum()
cpusum = pd.DataFrame(cpusum).reset_index()

# Plot total CPUs used in 20min before failure
fig5 = px.scatter(cpusum,
           x=cpusum['group'],
           y=cpusum['CPUS'],
           opacity=0.25,
           labels={'group':'Group Number', 'NODES':'Nodes used'},
           #title='It is unlikely that memory use is the direct cause of Unresponsive indicator',
           color_discrete_sequence=[yellow_color]
           )
# Calculate average y value
avg_y = cpusum['CPUS'].mean()

# Add horizontal line at average y
fig5.add_hline(
    y=avg_y,
    line_dash="dot",
    line_color=yellow_color,
    # label=dict(
    #     text=f"average number of CPUs used={avg_y:.2f}",
    #     textposition='start',
    #     font=dict(size=10, color=yellow_color),
    #     yanchor="top",
    # ),
)
# Change the background color
fig5.update_layout(plot_bgcolor=blue_color,
                      xaxis=dict(showgrid=False),
                      yaxis=dict(showgrid=False))
fig5.write_image('../assets/cpu_usage.jpeg')
fig5.show()

In [24]:
fig_multi = make_subplots(rows=5, cols=1, shared_xaxes=True)

fig_multi.add_trace(
    go.Scatter(x=totalmem['group'],
               y=totalmem['USEDMEM'],
               name='USEDMEM',
               opacity=0.5,
               marker=dict(color=yellow_color)
    ),
    row=1,
    col=1
)
fig_multi.update_layout(plot_bgcolor=blue_color,
                      xaxis=dict(showgrid=False),
                      yaxis=dict(showgrid=False))

fig_multi.add_trace(
    go.Scatter(x=totaldiffmem['group'],
               y=totaldiffmem['DIFFMEM'],
               name='DIFFMEM',
               opacity=0.5,
               marker=dict(color=yellow_color)
    ),
    row=2,
    col=1
)
fig_multi.update_layout(plot_bgcolor=blue_color,
                      xaxis=dict(showgrid=False),
                      yaxis=dict(showgrid=False))

fig_multi.add_trace(
    go.Scatter(x=jobcounts['group'],
               y=jobcounts['JOBCOUNT'],
               name='JOBCOUNT',
               opacity=0.5,
               marker=dict(color=yellow_color)
    ),
    row=3,
    col=1
)
fig_multi.update_layout(plot_bgcolor=blue_color,
                      xaxis=dict(showgrid=False),
                      yaxis=dict(showgrid=False))

fig_multi.add_trace(
    go.Scatter(x=nodesum['group'],
               y=nodesum['NODES'],
               opacity=0.5,
               name='NODES',
               marker=dict(color=yellow_color)
    ),
    row=4,
    col=1
)
fig_multi.update_layout(plot_bgcolor=blue_color,
                      xaxis=dict(showgrid=False),
                      yaxis=dict(showgrid=False))

fig_multi.add_trace(
    go.Scatter(x=cpusum['group'],
               y=cpusum['CPUS'],
               opacity=0.5,
               name='CPUs',
               marker=dict(color=yellow_color)
    ),
    row=5,
    col=1
)
fig_multi.update_layout(plot_bgcolor=blue_color,
                      xaxis=dict(showgrid=False),
                      yaxis=dict(showgrid=False))

# Update xaxis properties
fig_multi.update_xaxes(title_text="Group Number")

# Update yaxis properties
fig_multi.update_yaxes(
    title_text="Mem Used", 
    row=1, col=1, 
    title_standoff=20,
    title_font=dict(size=20, family='Arial'),
)

fig_multi.update_yaxes(
    title_text="Diff in Mem", 
    row=2, col=1, 
    title_standoff=20,
    title_font=dict(size=20, family='Arial'),
)

fig_multi.update_yaxes(
    title_text="# Jobs", 
    row=3, col=1, 
    title_standoff=20,
    title_font=dict(size=20, family='Arial'),
)

fig_multi.update_yaxes(
    title_text="# Nodes", 
    row=4, col=1, 
    title_standoff=20,
    title_font=dict(size=20, family='Arial'),
)

fig_multi.update_yaxes(
    title_text="# CPUs", 
    row=5, col=1, 
    title_standoff=20,
    title_font=dict(size=20, family='Arial'),
)

fig_multi.update_layout(#title_text="Memory, Job number, Node & CPU use unlikely to directly cause ACCRE to become unresponsive", 
                showlegend=False,
                margin=dict(l=100, r=50, t=100, b=100),
                height=1000)
# Change the background color
fig_multi.update_xaxes(showgrid=False)
fig_multi.update_yaxes(showgrid=False)
fig_multi.write_html('../assets/subplots.html')
fig_multi.write_image('../assets/subplots.jpeg')
fig_multi.show()

## Logistic Regression

In [25]:
# Broadened to 5 minute intervals in desired datetime column 'END' 
jobs_ce56['END'] = pd.to_datetime(jobs_ce56['END']).dt.floor('5min')
df = jobs_ce56[['END', 'JOBID', 'FAILED', 'USEDMEM', 'CPUS']]
reg_df = df.groupby('END').agg({'FAILED':'sum', 'JOBID': 'count', 'USEDMEM': 'mean', 'CPUS':'mean'}).reset_index()
reg_df = reg_df.rename(columns={'END': 'time', 'FAILED': 'slurm_crashes', 'JOBID': 'completed_jobs', 'USEDMEM': 'used_mem', 'CPUS': 'cpus'})
reg_df['fails'] = [1 if value > 0 else 0 for value in reg_df['slurm_crashes']]

# reg_df = pd.read_csv('../data/reg_df.csv')
reg_df.sort_values('time').isna().sum()
reg_df

,time,slurm_crashes,completed_jobs,used_mem,cpus,fails
0,2020-10-01 00:10:00,0,3,363.320000,1.000000,0
1,2020-10-01 00:15:00,0,3,489.283333,1.000000,0
2,2020-10-01 00:20:00,0,9,6.234444,4.000000,0
3,2020-10-01 00:25:00,0,18,77.647222,3.500000,0
4,2020-10-01 00:30:00,0,11,6.228000,4.454545,0
...,...,...,...,...,...,...
105505,2021-10-07 20:10:00,0,1,0.090000,1.000000,0
105506,2021-10-07 20:15:00,0,1,0.090000,1.000000,0
105507,2021-10-07 20:25:00,0,1,393.070000,1.000000,0
105508,2021-10-07 20:30:00,0,1,0.090000,1.000000,0


In [26]:
logit_model = smf.logit("fails ~ cpus + used_mem + completed_jobs", data = reg_df).fit()
logit_model.summary()

Optimization terminated successfully.
         Current function value: 0.102938
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  fails   No. Observations:               105457
Model:                          Logit   Df Residuals:                   105453
Method:                           MLE   Df Model:                            3
Date:                Sat, 07 Dec 2024   Pseudo R-squ.:                0.002116
Time:                        11:40:53   Log-Likelihood:                -10856.
converged:                       True   LL-Null:                       -10879.
Covariance Type:            nonrobust   LLR p-value:                 5.578e-10
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -3.8232      0.035   -110.448      0.000      -3.891      -3.755
cpus               0.0451      0.007      6.139      0.000       0.031       0.059
used_mem       -2.123e-05   3.93e-06     -5.404      0.000   -2.89e-05   -1.35e-05
completed_jobs    -0.0002      0.000     -1.331      0.183      -0.001       0.000
==================================================================================
"""

#### Logistic Regression function: 

$$\text{logit}(p) = -3.8232 + 0.0451(cpus) - 2.123e^-05(used\_mem) - 0.0002(completed\_jobs)$$

In [27]:
cpu_fit_df = pd.DataFrame({
    'used_mem': np.linspace(start = 0,
                            stop = 0,
                            num = 300),
    'cpus' : np.linspace(start = reg_df['cpus'].min(),
                            stop = reg_df['cpus'].max(),
                            num = 300),
    'completed_jobs': np.linspace(start = 0,
                            stop = 0,
                            num = 300)
})
cpu_fit_df.head(3)

,used_mem,cpus,completed_jobs
0,0.0,0.555556,0.0
1,0.0,0.981791,0.0
2,0.0,1.408027,0.0


In [28]:
cpu_fit_df['fit'] = logit_model.predict(cpu_fit_df)
cpu_fit_df

,used_mem,cpus,completed_jobs,fit
0,0.0,0.555556,0.0,0.021921
1,0.0,0.981791,0.0,0.022337
2,0.0,1.408027,0.0,0.022760
3,0.0,1.834262,0.0,0.023191
4,0.0,2.260498,0.0,0.023630
...,...,...,...,...
295,0.0,126.295058,0.0,0.866146
296,0.0,126.721293,0.0,0.868357
297,0.0,127.147529,0.0,0.870537
298,0.0,127.573764,0.0,0.872686


In [29]:
fig_cpu = px.scatter(cpu_fit_df,
           x=cpu_fit_df['cpus'],
           y=cpu_fit_df['fit'],
           opacity=0.5,
           labels={'cpus':'CPUs Used', 'fit':'P(SLURM Unresponsive)'},
        #    title='Logistic Curve: CPU usage',
           color_discrete_sequence=[yellow_color]
           )
fig_cpu
# Change the background color
fig_cpu.update_layout(plot_bgcolor=blue_color,
                      xaxis=dict(showgrid=False),
                      yaxis=dict(showgrid=False))
fig_cpu.write_html('../assets/cpu_logistic_plot.html')
fig_cpu.write_image('../assets/cpu_logistic.jpeg')
fig_cpu.show()

In [30]:
mem_fit_df = pd.DataFrame({
    'used_mem': np.linspace(start = reg_df['used_mem'].min(),
                            stop = reg_df['used_mem'].max(),
                            num = 300),
    'cpus' : np.linspace(start = 0,
                         stop = 0,
                         num = 300),
    'completed_jobs': np.linspace(start = 0,
                            stop = 0,
                            num = 300)
})
mem_fit_df.head()

,used_mem,cpus,completed_jobs
0,0.010000,0.0,0.0
1,1588.104515,0.0,0.0
2,3176.199030,0.0,0.0
3,4764.293545,0.0,0.0
4,6352.388060,0.0,0.0


In [31]:
mem_fit_df['fit'] = logit_model.predict(mem_fit_df)
mem_fit_df.head(3)

,used_mem,cpus,completed_jobs,fit
0,0.010000,0.0,0.0,0.021391
1,1588.104515,0.0,0.0,0.020696
2,3176.199030,0.0,0.0,0.020024


In [32]:
# Plot logistic curve for used memory
fig_mem = px.scatter(mem_fit_df,
           x=mem_fit_df['used_mem'],
           y=mem_fit_df['fit'],
           opacity=0.5,
           labels={'used_mem':'Memory Used', 'fit':'P(SLURM Unresponsive)'},
           title='Logistic Curve: Memory usage',
           color_discrete_sequence=[yellow_color]
           )
fig_mem
# Change the background color
fig_mem.update_layout(plot_bgcolor=blue_color,
                      xaxis=dict(showgrid=False),
                      yaxis=dict(showgrid=False))
fig_mem.write_image('../assets/mem_logistic.jpeg')
fig_mem

In [33]:
jobs_fit_df = pd.DataFrame({
    'used_mem': np.linspace(start = 0,
                         stop = 0,
                         num = 300),
    'cpus' : np.linspace(start = 0,
                         stop = 0,
                         num = 300),
    'completed_jobs': np.linspace(start = reg_df['completed_jobs'].min(),
                            stop = reg_df['completed_jobs'].max(),
                            num = 300)
})
jobs_fit_df.head()

,used_mem,cpus,completed_jobs
0,0.0,0.0,0.000000
1,0.0,0.0,7.391304
2,0.0,0.0,14.782609
3,0.0,0.0,22.173913
4,0.0,0.0,29.565217


In [34]:
jobs_fit_df['fit'] = logit_model.predict(jobs_fit_df)
jobs_fit_df

,used_mem,cpus,completed_jobs,fit
0,0.0,0.0,0.000000,0.021391
1,0.0,0.0,7.391304,0.021353
2,0.0,0.0,14.782609,0.021316
3,0.0,0.0,22.173913,0.021278
4,0.0,0.0,29.565217,0.021241
...,...,...,...,...
295,0.0,0.0,2180.434783,0.012697
296,0.0,0.0,2187.826087,0.012674
297,0.0,0.0,2195.217391,0.012652
298,0.0,0.0,2202.608696,0.012629


In [35]:
# Plot logistic curve for used memory
fig_jobs = px.scatter(jobs_fit_df,
           x=jobs_fit_df['completed_jobs'],
           y=jobs_fit_df['fit'],
           opacity=0.5,
           labels={'completed_jobs':'Completed Jobs', 'fit':'P(SLURM Unresponsive)'},
           title='Logistic Curve: Jobs',
           color_discrete_sequence=[yellow_color]
           )
fig_jobs
# Change the background color
fig_jobs.update_layout(plot_bgcolor=blue_color,
                      xaxis=dict(showgrid=False),
                      yaxis=dict(showgrid=False))
fig_jobs.write_image('../assets/jobs_logistic.jpeg')
fig_jobs

In [36]:
def logistic(x):
    return 1 / (1 + np.exp(-x))